In [37]:
# default_exp talker

# Talker

> Code for chatting with [tellyoulater](http://www.tellyoulater.com/)

In [38]:
# export
from fastcore.all import *
from bs4 import BeautifulSoup
from pathlib import Path
from functools import partial
from shutil import copyfile
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.keys import Keys
from itertools import count
from time import sleep
import http
import webbrowser
import flask
import os
import click
import pyperclip
import colorama
import pickle
import json

In [39]:
#hide
from nbdev.showdoc import *

In [40]:
encrypted_text="""202012090430-4ID_02OeiqruSuDOrKn2GvNsySCGPh5PnWHlur9wuxSvsovbbiUqvctKx3nb-s6
eGhF1exvfaxxLj7lOc9qR2qC8pQIXZz66KYjNc4tDDNF75pcnw2mpM294fJdJjr2mB2TBQI4M6WO
SF_WqKEUORmjwWABGiq0gOYb2FAdYQ_Hxy7GvN0QLswzZxu-JfjBECHfCo_8evHyDMaSdS-1KhnL
s7weLw===lYiH"""

previous_date_key="202012111930-4IEAJb8-vbHxgEHAojlb2xMAyJQYYmE29nb3dcCxxhEekj6f3dY_IVopegcSi-YmurGW2u9oNQk5e0TnIdZ0UWG0jX2tA7IFIq14tckSaZvoiasCxuIcfjonPUnOsJMIYmrYVQaxlvGDEOgPuIPmOPKKcxSoS7OXWm83qbw5C1ClwR0EFKqgkdqd1Q9OQsyUnDa-agpJIC5nwaKF2Q0YeqULnnnBU66rDoGLjZmzpKiOZUlq5-rb_jdjKRdm8Nnu6GiInUu0mXZP_eG-Ik8Jn-Xpb-2K4cKKwYWo1rH01m8vPhrCCUp84n6pU0ysIMTnsm2HizhRkcp3YXmXjPjeIGR6ONSw62J_0GxVKKkv86_KXPmgli-B77o2a418SHkxxT78J-3CKJPDS0KBTXVAiloW2cNwoUcJGQSgtDKIrektyd82g_NljPrxGIXOSwg4BsdKKZWs7pAc9k-l7zWzQubtIjd8bflFojqOTN60lrv_ZCBofUw2aas7WY1UYzfbkzwpN0ki83cG8vRI0YvDrWPqd15Zw===p_LL"

In [41]:
# export
def txt2encrypted_file(p:Path,txt:str):
    p=Path(p)/'encrypted_time_key.pkl'
    with open(p,'wb') as f: 
        pickle.dump(txt.replace('\n',''),f)

In [42]:
txt2encrypted_file('.',encrypted_text)

In [43]:
# export
def encrypted_file2txt(p:Path):
    if len(Path(p).parts)==0 or Path(p).parts[-1]!='encrypted_time_key.pkl':
        p=Path(p)/'encrypted_time_key.pkl'
    with open(p,'rb') as f: 
        return pickle.load(f)

In [44]:
encrypted_file2txt('.')

'202012090430-4ID_02OeiqruSuDOrKn2GvNsySCGPh5PnWHlur9wuxSvsovbbiUqvctKx3nb-s6eGhF1exvfaxxLj7lOc9qR2qC8pQIXZz66KYjNc4tDDNF75pcnw2mpM294fJdJjr2mB2TBQI4M6WOSF_WqKEUORmjwWABGiq0gOYb2FAdYQ_Hxy7GvN0QLswzZxu-JfjBECHfCo_8evHyDMaSdS-1KhnLs7weLw===lYiH'

## Key Setter

In [45]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver=webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
driver.get("http://www.tellyoulater.com")


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/play_it_later/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


In [46]:
# export
class HttpConnectionFailure(Exception):pass

def send_encryption_key(key,year=2020,month=12,day=10,hour=19,minute=30,max_tries=10):
    chrome_options=Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    with webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options) as driver:
        driver.get("http://www.tellyoulater.com")

        textbox=driver.find_element_by_id('intext')
        textbox.send_keys(key)
        
        button=driver.find_element_by_id('timeAM' if hour<12 else 'timePM')
        button.click()
        
        years_list=driver.find_element_by_id('yearShortcuts')
        years=[o.get_attribute('innerHTML') for o in years_list.find_elements_by_tag_name('a')]
        if str(year) not in years: click.echo(f'Please pick from these years: {years}')
        [o.click() for o in years_list.find_elements_by_tag_name('a') if o.get_attribute('innerHTML')==str(year)]
        
        if hour>12:hour-=12
        hours_list=driver.find_element_by_id('hours').find_elements_by_tag_name('a')
        hours=[int(o.get_attribute('innerHTML')[:-1]) for o in hours_list]
        if hour not in hours:click.echo(f'Please pick from these hours: {hours}')
        [o.click() for o in hours_list if int(o.get_attribute('innerHTML')[:-1])==hour]
        
        minutes_list=driver.find_element_by_id('minutes').find_elements_by_tag_name('a')
        minutes=[int(o.get_attribute('innerHTML')[1:]) for o in minutes_list]
        if minute not in minutes:click.echo(f'Please pick from these minutes: {minutes}')
        [o.click() for o in minutes_list if int(o.get_attribute('innerHTML')[1:])==minute]
        
        months_list=driver.find_element_by_id('monthShortcuts')
        months=[(i+1) for i,o in enumerate(months_list.find_elements_by_tag_name('a'))]
        if month not in months:
            click.echo(f'Please pick from these months: {month}')
        [o.click() for i,o in enumerate(months_list.find_elements_by_tag_name('a')) if (i+1)==month]
        
        date_picker=driver.find_element_by_id('datepicker')
        day_list=date_picker.find_element_by_tag_name('tbody').find_elements_by_tag_name('td')
        valid_day_list=[o.find_element_by_tag_name('a') for o in day_list if not in_('ui-state-disabled',o.get_attribute('class'))]
        days,day_elements=zip(*[(int(o.get_attribute('innerHTML')),o) for o in valid_day_list])
        if day not in days: click.echo(f'Please pick from these days in month {month}: {days}')
        [o.click() for i,o in zip(days,day_elements) if i==day]

        driver.find_element_by_id('encryptBtn').click()
        
        for i in count(0):
            res_code_element=driver.find_element_by_id('resCode').get_attribute('innerHTML').replace('\n','')
            if res_code_element=='':sleep(1)
            else:break
            if i==max_tries: raise HttpConnectionFailure('Failed to get res code')
                
        return res_code_element

In [47]:
queried_encryption_key=send_encryption_key(encrypted_text);queried_encryption_key

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/play_it_later/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


 
Please pick from these days in month 12: (11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)


'202012111930-4IEAJyH44BRycQO4aoQNXc2gM9pu1va-UtTSDNIdfVZOBe167yKY89Ai0cwsUmmJM3Y7BRa-z9Phdw-ia_AXM-PuBGJ-HJk7wLWWkTQuy8AiXdQulrjZnbBHJzECJ1N83kay-FPCjqxeL0JU0M2I3HiHhMYf7r22MNNh6269_gck5SvE-0-7RpksOJ0_30ZAcwUgnSI5LiPjd5lax2hpVl0ThJMt4d2EFCkAkG_rVDFVMZmIglJV95ZB6G7vxelVmdCjg1LD4lrn-DQuIIWVTITsU626uJ0HmYSHIkDaQ14X6z55TbvGTe9pqXL52w7fM8wVQKfQjYCQ2tUqSDluXkqyJOTq2dsT24txA49-9IvjEHUTSHe-U-hi7cxf3qADWgnfcwBYHjysn5DCmDAkLdaC2WeV7qGujKfhVRKkjsobqKozBnwSblqhbsdBpLus7WIcusLXR8-vk_xdGnJ4BeRQipJHM0AubeVB6SVKXvgVoskYPHamGixaJX7b3H4ifqVLdSGWS00RCe1_SEpPvz4I8SREA===1SJi'

In [48]:
txt2encrypted_file('.',queried_encryption_key)

## Key Getter

In [49]:
# export
def request_decryption_key(key,max_tries=10):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    with webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options) as driver:
        driver.get("http://www.tellyoulater.com/decrypt")

        textbox=driver.find_element_by_id('outtext')
        textbox.clear()
        textbox.send_keys(key)
        
        button=driver.find_element_by_id('decryptBtn')
        button.click()
        
        result_description=driver.find_element_by_id('msgDescription')
        click.echo(result_description.get_attribute('innerHTML'))
        
        for i in count(0):
            decrypted_msg=driver.find_element_by_id('decryptedMsg')\
                                     .get_attribute('innerHTML')\
                                     .replace('"','').replace('”','')\
                                     .replace('“','').replace('<br>','')
            if decrypted_msg=='':sleep(1)
            else:break
            if i==max_tries: raise HttpConnectionFailure('Failed to get decrypted_msg')
        return decrypted_msg
                

In [50]:
key=encrypted_file2txt('.')

In [51]:
decrypted_text=request_decryption_key(previous_date_key);decrypted_text

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/play_it_later/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


 
The message was encrypted to be inaccessible until 12/11/2020, 7:30:00 PM local time or Fri, 11 Dec 2020 19:30:00 GMT UTC


HttpConnectionFailure: Failed to get decrypted_msg

In [52]:
test_eq(decrypted_text,encrypted_text)

NameError: name 'decrypted_text' is not defined

In [54]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /opt/project/play_it_later/nbs/index.ipynb to README.md
Converted 00_encryption.ipynb.
Converted 01_talker.ipynb.
Converted index.ipynb.
converting: /opt/project/play_it_later/nbs/01_talker.ipynb
